## Load Environment Variables

In [2]:
'''import os 
from langchain_groq import ChatGroq
from dotenv import load_dotenv

groq_api_key=os.getenv("GROQ_API_KEY")
#groq_api_key'''

'import os \nfrom langchain_groq import ChatGroq\nfrom dotenv import load_dotenv\n\ngroq_api_key=os.getenv("GROQ_API_KEY")\n#groq_api_key'

In [3]:
'''model=ChatGroq(model_name="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
model'''

'model=ChatGroq(model_name="llama-3.3-70b-versatile",groq_api_key=groq_api_key)\nmodel'

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [5]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
import bs4

loader=WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2021-07-11-diffusion-models/",),

    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    ),
)

docs=loader.load()
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2021-07-11-diffusion-models/'}, page_content='\n\n      What are Diffusion Models?\n    \nDate: July 11, 2021  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n\n[Updated on 2021-09-19: Highly recommend this blog post on score-based generative modeling by Yang Song (author of several key papers in the references)].\n[Updated on 2022-08-27: Added classifier-free guidance, GLIDE, unCLIP and Imagen.\n[Updated on 2022-08-31: Added latent diffusion model.\n[Updated on 2024-04-13: Added progressive distillation, consistency models, and the Model Architecture section.\nSo far, I’ve written about three types of generative models, GAN, VAE, and Flow-based models. They have shown great success in generating high-quality samples, but each has some limitations of its own. GAN models are known for potentially unstable training and less diversity in generation due to their adversarial training nature. VAE relies on a sur

In [7]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
split=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=split,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever


VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001DD80ABBBE0>, search_kwargs={})

In [8]:
system_prompt=(
    "you are an assistant for question-answering tasks. " \
    "Use the following pieces of retrieved context to answer"
    "the question.If you don't know the answer,say that you"
    "don't know. Use three sentences maximum and keep the"
    "answer concise ."
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [9]:
rag_chain = (
    {
        "context": retriever,
        "input": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)


NameError: name 'model' is not defined

In [ ]:
response = rag_chain.invoke("What is a diffusion model?")
response


'A diffusion model is a type of model inspired by non-equilibrium thermodynamics that defines a Markov chain of diffusion steps to add random noise to data. It then learns to reverse the diffusion process to construct desired data samples from the noise. Diffusion models are learned with a fixed procedure and have a high-dimensional latent variable, similar to the original data.'

## Adding chat history 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser


In [ ]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question, "
    "which might reference context in the chat history, "
    "formulate a standalone question that can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [ ]:
# 8. Build history-aware retriever (LCEL)
# ===============================
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableLambda
)
from langchain_core.output_parsers import StrOutputParser

# Rewrite question using history
question_rewriter = (
    contextualize_q_prompt
    | model
    | StrOutputParser()
)


In [ ]:
# History-aware retriever
history_aware_retriever = RunnableLambda(
    lambda x: retriever.invoke(
        question_rewriter.invoke(x)
    )
)

In [ ]:
rag_chain = (
    {
        "context": history_aware_retriever,
        "input": RunnablePassthrough()
    }
    | qa_prompt
    | model
    | StrOutputParser()
)

In [ ]:
chat_history = []

response = rag_chain.invoke(
    {
        "input": "What is a diffusion model?",
        "chat_history": chat_history
    }
)

print(response)

# Example follow-up (history aware)
chat_history.extend([
    {"role": "human", "content": "What is a diffusion model?"},
    {"role": "assistant", "content": response}
])

response2 = rag_chain.invoke(
    {
        "input": "How do they work?",
        "chat_history": chat_history
    }
)

print(response2)

A diffusion model is a type of model inspired by non-equilibrium thermodynamics. It defines a Markov chain of diffusion steps to slowly add random noise to data and then learns to reverse the diffusion process to construct desired data samples from the noise. Unlike VAE or flow models, diffusion models are learned with a fixed procedure and have a high-dimensional latent variable.
Diffusion models work by defining a Markov chain of diffusion steps that slowly add random noise to data. They then learn to reverse this diffusion process to construct desired data samples from the noise. This process involves a forward diffusion process where Gaussian noise is added to the sample in multiple steps, and then a reverse process to remove the noise and generate the original data.
